<a target="_blank" href="https://colab.research.google.com/github/piavik/Prognosis/blob/dev/experiments/mlp_xgb.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
from google.colab import files
import pandas as pd
import io

# Загрузка файла
uploaded = files.upload()

# Чтение загруженного файла
for filename, content in uploaded.items():
    df = pd.read_csv(io.StringIO(content.decode('utf-8')))
    #print(df.head())

Saving internet_service_churn.csv to internet_service_churn (1).csv


In [10]:
# Create 'is_contract' column: 1 if 'reamining_contract' is not null, else 0
df['is_contract'] = df['reamining_contract'].notna().astype(int)

# Replace nulls in 'reamining_contract' with 0
df['reamining_contract'] = df['reamining_contract'].fillna(0)

# Define the desired column order
column_names = [
    'id', 'is_tv_subscriber', 'is_movie_package_subscriber', 'subscription_age',
    'bill_avg', 'reamining_contract', 'is_contract', 'service_failure_count',
    'download_avg', 'upload_avg', 'download_over_limit', 'churn'
]

# Reorder the DataFrame columns
df = df[column_names]

In [15]:
# Replace empty strings with NaN in 'download_avg' and 'upload_avg' columns
df[['download_avg', 'upload_avg']] = df[['download_avg', 'upload_avg']].replace('', np.nan)

# Drop rows where either 'download_avg' or 'upload_avg' is NaN
df.dropna(subset=['download_avg', 'upload_avg'], inplace=True)

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Заполнение пропущенных значений нулями
df.fillna(0, inplace=True)
numeric_columns = ['subscription_age', 'bill_avg', 'reamining_contract', 'service_failure_count', 'download_avg', 'upload_avg', 'download_over_limit']
# Стандартизация числовых переменных
scaler = StandardScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

In [21]:
from sklearn.neural_network import MLPClassifier

# 1. Подготовка данных
X = df.drop(['id', 'churn'], axis=1)
y = df['churn']

# Разделение на тренировочные и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. Создание модели MLP
mlp = MLPClassifier(hidden_layer_sizes=(128, 64, 32), max_iter=300, random_state=42)

# 3. Обучение модели
mlp.fit(X_train, y_train)

# 4. Оценка модели
y_pred = mlp.predict(X_test)

# Оценка метрик
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 score: {f1:.4f}")


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy: 0.9308
Precision: 0.9430
Recall: 0.9336
F1 score: 0.9383


In [22]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 1. Подготовка данных
X = df.drop(['id', 'churn'], axis=1)
y = df['churn']

# Разделение на тренировочные и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. Создание модели XGBoost
#model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)
model = xgb.XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=6, subsample=0.8, colsample_bytree=0.8, random_state=42)

# 3. Обучение модели
model.fit(X_train, y_train)

# 4. Оценка модели
y_pred = model.predict(X_test)

# Оценка метрик
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 score: {f1:.4f}")


Accuracy: 0.9440
Precision: 0.9590
Recall: 0.9409
F1 score: 0.9499
